In [ ]:
# local dependencies
from load import *
from plots import *
from clustering_helpers import *
from constants import *

### Plotting silhouette, Davis-Boudin index and score of agglomerative, kmeans and spectral clustering methods on the pdx dataset

In [ ]:
pdx = load_pdx()
X = pdx.drop("label", axis=1)
pdx_labeled = pdx.label

test_all_methods(X, pdx_labeled, True)

### Plotting the silhouettes of the agglomerative clustering on the patients dataset with various combinations of affinities and linkages

In [ ]:
patients2 = load_patients2()

In [ ]:
test_all_methods(patients2, None)